In [6]:
from pyspark.sql import SparkSession
import pymongo

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook2").\
        master("local[*]").\
        config("spark.executor.memory", "1g").\
        config("spark.mongodb.input.uri","mongodb://172.22.0.2:27017/student.grades").\
        config("spark.mongodb.output.uri","mongodb://172.22.0.2:27017/student.grades").\
        config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
        getOrCreate()

logger = spark._jvm.org.apache.log4j
logger.LogManager.getRootLogger().setLevel(logger.Level.FATAL)

    
# read from MongoDB collection
df = spark.read.format("mongo").load()

In [7]:
df.show()

+--------------------+-----+--------+---------+
|                 _id|grade|    name|telephone|
+--------------------+-----+--------+---------+
|{64656cd842b78f63...|   30|    raul|     null|
|{64656d0742b78f63...|   55|    paul|  1234567|
|{64656e0e42b78f63...|   88| charles|     null|
|{64656f2342b78f63...|   99|    fifi|     null|
|{64656f2342b78f63...|   75|pinochio|     null|
|{6465a5a2affbd2fb...|   70| charles|     null|
+--------------------+-----+--------+---------+



In [8]:
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- grade: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- telephone: integer (nullable = true)



In [9]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F
movAvg = df.withColumn("gradeaverage", F.avg("grade")
.over( Window.partitionBy("name").rowsBetween(-1,1)) )

In [10]:
movAvg.show()

+--------------------+-----+--------+---------+------------+
|                 _id|grade|    name|telephone|gradeaverage|
+--------------------+-----+--------+---------+------------+
|{64656e0e42b78f63...|   88| charles|     null|        79.0|
|{6465a5a2affbd2fb...|   70| charles|     null|        79.0|
|{64656f2342b78f63...|   99|    fifi|     null|        99.0|
|{64656d0742b78f63...|   55|    paul|  1234567|        55.0|
|{64656f2342b78f63...|   75|pinochio|     null|        75.0|
|{64656cd842b78f63...|   30|    raul|     null|        30.0|
+--------------------+-----+--------+---------+------------+

